In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder

quotes_1k = pd.read_csv('datasets/insparationalQuotes1K.csv')
quotes_1k = quotes_1k[['Quote', 'Category']]
quotes_1k.columns = ['quote', 'category']

#claning
def clean_text(text):
    if pd.isnull(text):
        return ""
    text = str(text).strip()             
    text = ' '.join(text.split())         #extra spaces
    text = text.lower()                  #lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  #non-alphabetical characters
    return text
quotes_1k['quote'] = quotes_1k['quote'].apply(clean_text)
quotes_1k['category'] = quotes_1k['category'].str.strip().str.lower()

#shuffle dataset
quotes_1k = quotes_1k.sample(frac=1, random_state=10).reset_index(drop=True)

#sampling (1000)
sampled_quotes = quotes_1k.sample(n=1000, random_state=10).reset_index(drop=True)

#label encoding
le = LabelEncoder()
sampled_quotes['category_encoded'] = le.fit_transform(sampled_quotes['category'])

import pickle
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

#save dataset
sampled_quotes.to_csv('datasets/cleanedDataSecondModel.csv', index=False)
sampled_quotes.head()


,quote,category,category_encoded
0,a generous heart kind speech and a life of ser...,stewardship,104
1,education breeds confidence,confidence,18
2,a smile is a curve that sets everything straight,smile,100
3,it is not joy that makes us grateful it is gra...,gratitude,45
4,we say things will turn out but no its what we...,stewardship,104


In [2]:
# bersigin negative.txt
import pandas as pd
import re

with open('datasets/negative.txt', 'r', encoding='utf-8') as f:
    raw_negative = f.readlines()

def clean_text(text):
    text = str(text).strip()
    text = ' '.join(text.split())
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  #punctuation
    return text

#buang baris kosong
cleaned_negative = [clean_text(line) for line in raw_negative if line.strip() != '']

negative_df = pd.DataFrame(cleaned_negative, columns=['quote'])
negative_df.insert(0, 'label', 0)  #label 0 non-motivational
negative_df = negative_df.sample(n=1000, random_state=42).reset_index(drop=True)

positive_df = pd.read_csv('datasets/cleanedDataSecondModel.csv')
positive_df = positive_df[['quote']]
positive_df['quote'] = positive_df['quote'].astype(str).apply(clean_text)
positive_df.insert(0, 'label', 1)  #label 1 motivational



#gabungan negative&positive
binary_df = pd.concat([positive_df, negative_df], ignore_index=True)
binary_df = binary_df.sample(frac=1, random_state=42).reset_index(drop=True)
#save dataset
binary_df.to_csv('datasets/binaryDataFirstModel.csv', index=False)
print(binary_df.head())


   label                                              quote
0      0  children  christian or otherwise  deserve to h...
1      1  what lies behind you and what lies in front of...
2      0  none of this socalled satire has any sting to ...
3      1  compassion is the sometimes fatal capacity for...
4      0  like all abstract art  the film does not make ...
